In [771]:
import pandas as pd
import numpy as np
from collections import Counter

# Input
return 
- training data
- intermediate data
- test data
- sentence to decode

In [772]:
train = pd.read_table("/Users/amyburkhardt/Dropbox/NLP Readings/hw 1/rand_training.txt",'\t', 
                      header=None, 
                      skip_blank_lines=False, 
                      keep_default_na = False,
                      names = ['word_Num', 'word', 'tag'])

In [773]:
train

,word_Num,word,tag
0,1,i,PRP
1,2,'d,MD
2,3,like,VB
3,4,to,TO
4,5,go,VB
5,6,to,IN
6,7,a,DT
7,8,fancy,JJ
8,9,restaurant,NN
9,10,.,.


In [499]:
new_sent = ['the', 'dog', 'ate', 'the', 'food']

In [428]:
text_strings

['i', "'d", 'like', 'to', 'go', 'to', 'a', 'fancy', 'restaurant', '.']

# Create Fixed Vocabulary and Tag Lists
return 
- tag list
- vocabuarly list called vocabulary
- events list

In [774]:
tags = ['CC', 'CD',
        'DT',
        'EX',
        'FW',
        'IN', 
        'JJ', 'JJR', 'JJS',
        'LS', 
        'MD',
        'NN', 'NNS', 'NNP', 'NNPS',
        'PDT', 'POS', 'PRP', 'PRP$',
        'RB', 'RBR', 'RBS', 'RP',
        'SYM', 
        'TO', 
        'UH', 
        'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ',
        'WDT', 'WP', 'WP$', 'WRB', 
        '$', '#', '"', '(', ')', ',', '.', ':'
       ]

In [775]:
def ngram_dict(data, ngrams = "tag_word"):
    """
    Creates dict of ngrams (key) and count (value). 
    
    Arguments: 
        data: DataFrame with 'tag' and 'word' colum
        negrams: denote type of ngram (unigram or bigram) and if want words or tags: word_word or tag_word
    Returns:
        A dict where key is either a unigram or a bigram tuple, and value is the count of the ngrams
    """
    if ngrams == "tag_tag":     
        col_1 = data['tag']
        col_2 = col_1[1:col_1.shape[0]]
        ngram_count = list(zip(col_1, col_2))
        ngram_count = dict(Counter(ngram_count))
        ngram_count[('', col_1[0])] += 1

    
    if ngrams == "tag_word": # not really bi-grams, just getting count of tag,word
        col_1 = data['word']
        col_2 = data['tag']
        ngram_count = list(zip(col_1, col_2))
        ngram_count = dict(Counter(ngram_count))
            
    if ngrams == 'tag': 
        ngram_count = dict(Counter(data.tag))      
        
    if ngrams == 'word': 
        ngram_count = dict(Counter(data.word))
            
    return ngram_count

## Get fixed vocabulary; identify which words will be considered UNKs

In [776]:
# get words that we will call unknowns, and replace these instances in the dataframe
unigrams = ngram_dict(train, "word")
unknowns = { key:value for key, value in unigrams.items() if value < 2 }
unknowns = unknowns.fromkeys(unknowns, 'UNK')
# replace words that appear less than three times with UNK in training data
train['word'] = train['word'].replace(unknowns)
#get list of vocabulary 
vocab = ngram_dict(train, "word")
vocabulary = list(vocab.keys())
vocabulary.remove('') # remove spaces

# Return index of the words in the new sentece from the fixed vocabuarly

In [777]:
events = []
for word in new_sent:
    print(word)
    try: 
        events.append(vocabulary.index(word))
    except: events.append(vocabulary.index('UNK'))
events

won
thai
cuisine
yorkshire
fish
and
chips
zachary
's
chicago
yorkshire
fish
and
chips
.


[443, 451, 82, 385, 762, 1089, 1157, 907, 574, 144, 385, 762, 1089, 1157, 992]

# Compute Transition and Observation Matrices
return
-tran matrix
-observation matrix

In [778]:
def compute_transition_matrix (tags, bigram_counts, unigram_counts):
    """
    Compute probabilities for the transition matrix (len(tags)+1 x len(tags))
    
    Arguments: 
        tags: POS tags (that may or may not appear in training data)
        bigram_counts: count of bigrams of POS tags in training data (used for numerator)
        unigram_counts: count of unigram POS tag in training data (used for denominator)
        
    Returns: 45 x 44 matrix of transition probabilities for all possible POS tags
    
    """

    transition = [] # list of transition probabilities 
    
    # first compute the starting probabilities 

    for x in tags: 
            pair = ('',x) # here the period denotes the start of a sentence. Not very confident about this
            denominator = unigram_counts[''] + len(tags)
            try: 
                 numerator = bigram_counts[pair] + 1 
            except:
                 numerator = 1
            transition.append(numerator / denominator)


    # then compute everything else 
    
    for x in tags:
        for y in tags:
            pair = (x,y)
            try:
                denominator = unigram_counts[x] + len(tags)
            except: 
                denominator = len(tags)
            try: 
                numerator = bigram_counts[pair] + 1 
            except:
                numerator = 1 
            transition.append(numerator / denominator)
   
    
    transition = np.array(transition)
    tran_matrix = transition.reshape(len(tags)+1, len(tags))
    
    return tran_matrix

In [779]:
bigram_tag_counts = ngram_dict(train, "tag_tag")
unigram_tag_counts = ngram_dict(train, "tag")
transitions = compute_transition_matrix (tags, bigram_tag_counts, unigram_tag_counts)
np.sum(transitions, axis = 1) # confirm that most rows sum closely to 1

array([ 1.00007896,  1.        ,  1.        ,  0.9923554 ,  1.        ,
        0.94901961,  1.        ,  0.9950701 ,  1.        ,  1.        ,
        0.84615385,  1.        ,  0.97792019,  0.99894715,  0.97271268,
        1.        ,  1.        ,  1.        ,  0.99980607,  1.        ,
        0.98761545,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  0.99947871,  1.        ,  0.98099762,
        0.99438202,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  0.00347387,  1.        ])

In [780]:
def compute_observation_matrix (tags, vocabulary, bigram_counts, unigram_counts):
    """
    Compute probabilities for the observation matrix (tags, vocabulary)
    
    Arguments: 
        tags: POS tags (that may or may not appear in training data)
        vocabulary: words that appear in the training set. Any words that appear less than 2 times = UNK
        bigram_counts: count of bigrams of (tag, word) (used for numerator)
        unigram_counts: count of unigram POS tag in training data (used for denominator)
        
    Returns: len(tags) x len(vocabulary) matrix of transition probabilities for all possible POS tags
    
    """

    observations = [] # list of observation likelihoods
    for x in tags: 
        for y in vocabulary:
            pair = (y, x)
            try: 
                denominator = unigram_counts[x] + len(vocabulary)
            except: 
                 denominator = len(vocabulary)
            try: 
                 numerator = bigram_counts[pair] + 1
            except: numerator = 1
            observations.append(numerator / denominator)
            
    observations = np.array(observations)
    obs_matrix = observations.reshape(len(tags),len(vocabulary))  
    return obs_matrix

In [781]:
bigram_counts = ngram_dict(train, "tag_word")
unigram_counts = ngram_dict(train, "tag")
observations = compute_observation_matrix(tags, vocabulary, bigram_counts, unigram_counts)

In [782]:
np.sum(observations, axis = 1) # confirm that most rows sum closely to 1

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.])

# Viterbi Algorithm
return
-predicted POS tags

In [783]:
def viterbi (transition, observations, events):
    """ Computes sequnce of hidden states, given observed events.
    Arguments: 
        transition: transition matrix with start probabilites as first row
        observations: observation liklihood matrix, with states as rows, and vocabulary as columns
        events: sequence of observed events
        
    Returns: 
        generator, which yields the states
    """
    
    n_states = transition.shape[1]
    n_events = len(events)
    v = np.zeros((n_states, n_events))
    bp = v.copy()
    
    # initialization step
    for s in range(n_states):
        v[s,0] = tran[0,s] * observations[s, events[0]]

    # induction step
    for t in range (1, n_events):
        for s in range(n_states):
            tmp = []
            for s_prime in range (n_states): 
                prev_t = v[s_prime, t-1]
                tran_s_prime_to_s = tran[s_prime + 1, s]
                obser_s_given_t = observations[s, events[t]]
                tmp.append(prev_t * tran_s_prime_to_s * obser_s_given_t) # still need to changet thos to adding logs
            # now that all interim probabilities have been computed for given state, get max
            # and also store the index of the argmax
            v[s,t] = max(tmp) # log will be negative; so insetad of 
            bp[s,t] = np.argmax(tmp) # take argmin 

    # termination step
    q = np.argmax(v[:, n_events-1]) # want to get the argmax of the final time -- it will return a state index

    # back reference step 
    for i in reversed(range(n_events)):
        yield q
        q = int(bp[q,i])


    

In [784]:
def get_sequence(viterbi_gen, names_events):
    """ translate viterbi generater into a sequence of state anme
    """
    sequence = []
    for state in viterbi_gen:
        name = names_events[state]
        sequence.insert(0, name)
        
    return(sequence)

In [785]:
tagger = viterbi(transitions, observations, events)

In [786]:
get_sequence(tagger, tags)

/Users/amyburkhardt/Documents/NLP/venv/lib/python3.5/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in log


['NNPS',
 'CC',
 'CC',
 'CC',
 'CC',
 'CC',
 'CC',
 'CC',
 'CC',
 'CC',
 'CC',
 'CC',
 'CC',
 'CC',
 'CC']

In [743]:
sentences = train['word'].tolist()

In [744]:
def sent(seq, sep):
    g = []
    for el in seq:
        if el == sep:
            yield g
            g = []
        g.append(el)
    yield g
    

result = list(sent(sentences, ''))

In [745]:
def get_events(new_sent):
    events = []
    for word in new_sent:
        try: 
            events.append(vocabulary.index(word))
        except: events.append(vocabulary.index('UNK'))
    return events

In [752]:
all_pos = []
counter = 0
for new_sent in result:
    if counter > 0: 
        new_sent.pop(0)
    tagger = viterbi(transitions, observations, get_events(new_sent))
    print(new_sent)
    sequence = get_sequence(tagger, tags)
    sequence.insert(len(sequence), '') #add space at the end
    all_pos.append(sequence)
    counter += 1

In [747]:
flat_list = [item for sublist in all_pos for item in sublist]

In [748]:
df = pd.DataFrame({'col':flat_list})

In [750]:
df = pd.DataFrame({'col':flat_list})
train['tag'] = df

train.to_csv("/Users/amyburkhardt/Dropbox/NLP Readings/hw 1/out_train_922.txt", sep='\t', index=False, header=False)


## to do list
1. make sure I am adding the log of the probabilities, instead of multiplying probabilties in viterbi (and that it still works)
2. TONIGHT: write function to print tags from get_sequence into .txt file in a format that can then read in and evaluated by eval.py
3. place code in final .py file (make it pretty if have time)
4. upon working code, evaluate both training and intermediate test data files
5. recompute transition and observation tables with entire training set
6. MONDAY NIGHT: wait for final test set to run through system
7. MONDAY NIGHT: write up report


In [763]:
test = pd.read_table("/Users/amyburkhardt/Dropbox/NLP Readings/hw 1/rand_test.txt",'\t', 
                      header=None, 
                      skip_blank_lines=False, 
                      keep_default_na = False,
                      names = ['word_Num', 'word','tag'])

In [764]:
sentences = test['word'].tolist()

In [765]:
def sent(seq, sep):
    g = []
    for el in seq:
        if el == sep:
            yield g
            g = []
        g.append(el)
    yield g
    

result = list(sent(sentences, ''))

In [766]:
def get_events(new_sent):
    events = []
    for word in new_sent:
        try: 
            events.append(vocabulary.index(word))
        except: events.append(vocabulary.index('UNK'))
    return events

In [767]:
all_pos = []
counter = 0
for new_sent in result:
    if counter > 0: 
        new_sent.pop(0)
    tagger = viterbi(transitions, observations, get_events(new_sent))
    #print(new_sent)
    sequence = get_sequence(tagger, tags)
    sequence.insert(len(sequence), '') #add space at the end
    all_pos.append(sequence)
    counter += 1

In [768]:
flat_list = [item for sublist in all_pos for item in sublist]

In [769]:
df = pd.DataFrame({'col':flat_list})

In [770]:
df = pd.DataFrame({'col':flat_list})
test['tag'] = df

test.to_csv("/Users/amyburkhardt/Dropbox/NLP Readings/hw 1/out_test_922.txt", sep='\t', index=False, header=False)
